<a href="https://colab.research.google.com/github/omarjilanijidan/Iris-Cataract-Detection-From-Fundus-Images/blob/main/Augmentation_data_save_to_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
import os
import numpy as np
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model

In [2]:
# Authenticate and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
image_size = 224
dataset_dir = ""

In [3]:
def create_dataset(image_category, label):
    dataset = []
    labels = []
    image_folder = os.path.join(dataset_dir, image_category)
    for img in tqdm(os.listdir(image_folder)[:1000]):
        image_path = os.path.join(image_folder, img)
        try:
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            if image is None:
                raise Exception("Failed to load image data")
            image = cv2.resize(image, (image_size, image_size))
            image = image / 255.0
            dataset.append(image)
            labels.append(label)
        except Exception as e:
            print(e)
    return np.array(dataset), np.array(labels)

In [6]:
cataract_images, cataract_labels = create_dataset("/content/drive/MyDrive/Papers/Dataset Fundus Image/Cataract", 1)
normal_images, normal_labels = create_dataset("/content/drive/MyDrive/Papers/Dataset Fundus Image/NDG", 0)

100%|██████████| 1000/1000 [01:32<00:00, 10.75it/s]


In [7]:
X = np.concatenate((cataract_images, normal_images[:1000]), axis=0)
y = np.concatenate((cataract_labels, normal_labels[:1000]), axis=0)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Print number of images used for training and testing
print(f"Number of images used for training: {len(X_train)}")
print(f"Number of images used for testing: {len(X_test)}")

Number of images used for training: 1600
Number of images used for testing: 400


In [10]:
# Pre-augmentation
augmented_train_images = []
augmented_train_labels = []
augmented_test_images = []
augmented_test_labels = []

# Augmentation for training data
for image, label in zip(X_train, y_train):
    # Random rotation
    random_rotation = random.uniform(-20, 20)
    rotated_image = cv2.getRotationMatrix2D((image_size / 2, image_size / 2), random_rotation, 1)
    rotated_image = cv2.warpAffine(image, rotated_image, (image_size, image_size))
    augmented_train_images.append(rotated_image)
    augmented_train_labels.append(label)

    # Random horizontal flip
    if random.random() > 0.5:
        flipped_image = cv2.flip(image, 1)
        augmented_train_images.append(flipped_image)
        augmented_train_labels.append(label)

# Augmentation for testing data
for image, label in zip(X_test, y_test):
    # Random rotation
    random_rotation = random.uniform(-20, 20)
    rotated_image = cv2.getRotationMatrix2D((image_size / 2, image_size / 2), random_rotation, 1)
    rotated_image = cv2.warpAffine(image, rotated_image, (image_size, image_size))
    augmented_test_images.append(rotated_image)
    augmented_test_labels.append(label)

    # Random horizontal flip
    if random.random() > 0.5:
        flipped_image = cv2.flip(image, 1)
        augmented_test_images.append(flipped_image)
        augmented_test_labels.append(label)

In [11]:
# Concatenate augmented images and labels with original data for training set
X_train = np.concatenate((X_train, np.array(augmented_train_images)), axis=0)
y_train = np.concatenate((y_train, np.array(augmented_train_labels)), axis=0)

# Concatenate augmented images and labels with original data for testing set
X_test = np.concatenate((X_test, np.array(augmented_test_images)), axis=0)
y_test = np.concatenate((y_test, np.array(augmented_test_labels)), axis=0)

In [12]:
# Define file paths for saving data
save_path = '/content/drive/MyDrive/Papers/Dataset Fundus Image/augmented-data'
train_images_path = os.path.join(save_path, 'X_train.npy')
train_labels_path = os.path.join(save_path, 'y_train.npy')
test_images_path = os.path.join(save_path, 'X_test.npy')
test_labels_path = os.path.join(save_path, 'y_test.npy')



# Save training and testing data to Google Drive
np.save(train_images_path, X_train)
np.save(train_labels_path, y_train)
np.save(test_images_path, X_test)
np.save(test_labels_path, y_test)

In [13]:
# Print number of images used for training and testing
print(f"Number of images used for training: {len(X_train)}")
print(f"Number of images used for testing: {len(X_test)}")

Number of images used for training: 3997
Number of images used for testing: 1004
